# Heart disease training model

In [33]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

Importar el csv:

In [34]:
class HeartDataset(Dataset):
    def __init__(self, file, transform=None, target_transform=None):
        data = pd.read_csv(file)
        self.X = torch.tensor(data.iloc[:, :-1].values, dtype=torch.float32)
        self.y = torch.tensor(data.iloc[:, -1].values, dtype=torch.float32).unsqueeze(1)

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        features = self.X[index]
        label = self.y[index]

        if self.transform:
            features = self.transform(features)
        if self.target_transform:
            label = self.target_transform(label)

        return features, label
    

dataset = HeartDataset(file='data/heart.csv')

Dividir les dades en train (80%) i test (20%) de forma aleatòria fent servir ``random_split``

In [35]:
dataset_len = len(dataset)
train_size = int(dataset_len * 0.8) # Les dades d'entrenament son el 80% del dataset
test_size = dataset_len - train_size # Les dades de test son el 20% restant
train_data, test_data = random_split(dataset, [train_size, test_size])

DataLoader:

In [36]:
bs = 64
train_dataloader = DataLoader(train_data, batch_size=bs, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=bs, shuffle=True)

Dispositiu que utilitzarem (GPU):

In [37]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


Definim el model neuronal:

In [38]:
class HeartModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(13,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,1)
        )
    
    def forward(self,x):
        logits = self.linear_relu_stack(x)
        return torch.sigmoid(logits).squeeze()
    
model = HeartModel()
print(model)

HeartModel(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)
